# __Step 7.1a: Testing nominatim container__

## ___Note___

This has been a hell of a ride for three weeks. The main obstacles are:
- Familiarity with docker
- Familiarity with nominatim server, particularly on how to get it up and running with very large osm.pbf files.
- Familiarity with nominatim docker, particularly on how export/import containers, setup persistent container (so the database is mounted), export/important a persistent database.

What I did wrong:
- When I run the container, I did not use the persistent container option. 
- I have imported the exported volume wrongly.
- Chased down the wrong directions (thought it did not work because of docker, nominatim, database, permission or their combinations) - but turned out it is simply that I import the volume wrong with the wrong directory structure.

### Nominatim query and installation

- [This tutorial](https://www.linkedin.com/pulse/geocoding-geopy-your-own-nominatim-server-chonghua-yin?trk=related_artice_Geocoding%20with%20GeoPy%20and%20Your%20Own%20Nominatim%20Server_article-card_title).
- Installation: quite challenging, did not finish. Use [Docker image](https://github.com/mediagis/nominatim-docker/blob/master/4.2/README.md) instead.
- Query through the Nominatim service resulted in time out. So use a docker version
  - Intial docker run: found out that only Monaco was included. 

### Nomanatim docker

- Look for ways to load the whole world.
  - Found and downloaded `planet-230213.osm.pbf` from [OpenStreeMap](https://planet.openstreetmap.org/).
  - [Not needed] Figure out [how to transfer files between local host and container](https://www.edureka.co/community/10534/copying-files-from-host-to-docker-container) and move all pbf files into docker container and commit the image with files.
  - Found out docker bind mount option (-v) so local file system can be mounted for use in the container.
  - Run out of memory
- Do each continent instead
  - Europe (26.4G):
    - It ends with error.
    - Try merge multiple smaller files instead with `osmium`, see [this post](https://gis.stackexchange.com/questions/242704/merging-osm-pbf-files) on install and merging osm.pbf files.
  - North America (12.2G): 
    - osm2pgsql took 8240s (2h 17m 20s) overall
- Alternative docker installation for WSL:
  - See [WSL doc](https://learn.microsoft.com/en-us/windows/wsl/tutorials/wsl-containers)

### Back/restore/migrate notes

Some info gathered:
- [Back up/restore/migrate data volume](https://docs.docker.com/storage/volumes/#back-up-restore-or-migrate-data-volumes): not useful... initially, but it is helpful.
- [How to copy docker volume from one machine to another?](https://stackoverflow.com/questions/42973347/how-to-copy-docker-volume-from-one-machine-to-another): Not what I want. turn out this is helpful.
- [Look at the 1st answer, 3rd solution](https://stackoverflow.com/questions/45714456/how-to-migrate-docker-volume-between-hosts)
  - The copy volume to local storage bit works.
  - But the import part does not.
- [Docker create volume](https://docs.docker.com/engine/reference/commandline/volume_create/)
  - And look into this that does [volume mounting](https://stackoverflow.com/questions/49950326/how-to-create-docker-volume-device-host-path)
- Ok, [this video](https://duckduckgo.com/?q=docker+import+local+file+as+a+volume&t=newext&atb=v314-1&iax=videos&ia=videos&iai=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DZEy8iFbgbPA) is extremely helpful. Circle back to the 1st option. See his [boilerplates](https://github.com/christianlempa/boilerplates)
  - [This post](https://stackoverflow.com/questions/38298645/how-should-i-backup-restore-docker-named-volumes) has good explanations. But this is outdated.

Ok, what happened here is that:
- The export/backup bit works file. The tar archive is created without problem, although there are some minor differences in the files. Not sure where they come from.
- Importing has been the major problem. The main issue is that I did not check the file structure after import after using some of the example command lines. This created all kinds of issues.
  - I have been chasing down the rabbit hole thinking it is a problem with Nominatim startup script, docker container used (original, saved, or imported), the differences in files.
  - But after I figure out how to look into docker volumes in `/var/lib/docker/volumes/` and compare them with `diff -qr`, I found that the imported folder is compeletely wrong.
- Works now.


In [ ]:
# Install and start docker daemon:
'''bash
sudo apt-get remove docker docker-engine docker.io containerd runc

sudo apt-get update
sudo apt-get upgrade
sudo apt install docker.io
sudo dockerd
'''

# Test Nominatim docker image:
'''bash
sudo groupadd docker
sudo usermod -aG docker shius
#su -s shius # <---- Run into error with this line
docker run hello-world # testing
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/monaco-latest.osm.pbf \
  -e REPLICATION_URL=https://download.geofabrik.de/europe/monaco-updates/ \
  -p 8080:8080 \
  --name nominatim \
  mediagis/nominatim:4.2
'''

# Testing, in browser:
#-http://localhost:8080/search.php?q=avenue%20pasteur

'bash\nsudo groupadd docker\nsudo usermod -aG docker shius\nsu -s shius\ndocker run hello-world # testing\ndocker run -it   -e PBF_URL=https://download.geofabrik.de/europe/monaco-latest.osm.pbf   -e REPLICATION_URL=https://download.geofabrik.de/europe/monaco-updates/   -p 8080:8080   --name nominatim   mediagis/nominatim:4.2\n'

## ___Approach going forward___

### Initial server run

- Use the FREEZE parameter
  - See [this](https://nominatim.org/release-docs/latest/admin/Advanced-Installations/): database looses the ability to receive further data updates but the resulting database is only about a third of the size of a full database
- Set up local bind mount to /nominatim/data
- Set up volumne data_andorra

In [ ]:
docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v data_andorra:/var/lib/postgresql/14/main \
  -v /home/shinhan/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_andorra \
  mediagis/nominatim:4.2

docker volume ls
docker volume inspect data_andorra

# Committ/save/export container
docker commit nominatim_andorra nominatim_andorra-commit:latest
docker save nominatim_andorra-commit > nominatim_andorra-save.tar

#https://www.techrepublic.com/article/how-to-exportimport-containers-with-docker/
docker export nominatim_andorra > nominatim_andorra-export.tar

### Exporting volume with a temp container

In [ ]:
# Nominatim example
docker run --rm \
  --volumes-from nominatim_andorra \
  -v $PWD:/backup \
  busybox tar cvf /backup/backup-andorra.tar /var/lib/postgresql/14/main

# stop nominatim run
docker rm -f nominatim_andorra

### Import archived volume

- Create a container (`nomi_restore`) with the postgresql data directory bound to it
- Import archived volume using `nomi_restore`
- Create an empty volume `data_andorra_restore`
- Move files from the volume created with `nomi_restore` (because it has a long ass name).
- Remove the original volume.

In [ ]:
# For nominatim
# temp container: nomi_restore
docker run -v /var/lib/postgresql/14/main --name nomi_restore ubuntu /bin/bash

docker run --rm \
  --volumes-from nomi_restore \
  -v $PWD:/backup \
  bash -c "tar xvf /backup/backup-andorra.tar"

docker volume ls
docker volume create --name data_andorra_restore

docker run --rm -it \
  -v 3ae370e6338ab458bcbed92081d60cb332a65128dfd78397ae9db0be786cfd46:/from \
  -v data_andorra_restore:/to \
  bash -c "cd /from ; cp -av . /to"

docker ps -a  # Check if there is container using the old volume
docker rm -f 33fa297caa91
docker volume rm 9333011fc3975fe5ce824f9011bf4e3820b28eb7c83541badbdeb15781511277

### Start nominatim using the imported volume

- Use the Equatorial Guinea data to run nominatim
- Use the original nominatim container
  - Did not test if the saved image, exported container work or not
- Tested with:
  - Shoud not work: http://localhost:8080/search.php?q=Malabo
    - Even though the PBF_PATH point to a Eq Gui file and Malabo is the capital for the country, it should not return result becuase Andorra database is used.
  - Shoud work: http://localhost:8080/search.php?q=aNDORRA
    - Ok...

In [ ]:
docker run -it \
  -e PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -p 8080:8080 \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  -v /home/shinhan/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_andorra2 \
  mediagis/nominatim:4.2

docker rm nominatim_andorra2

## ___Testing: Initial nominatim setup___

#### [mediagis/nominatim-docker](https://github.com/mediagis/nominatim-docker/blob/master/4.2/README.md)

Starting out with this, realize that different regions need to be downloaded.

In [ ]:
'''
TESTING
---
# Start Nominatim container in another terminal
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/monaco-latest.osm.pbf \
  -e REPLICATION_URL=https://download.geofabrik.de/europe/monaco-updates/ \
  -p 8080:8080 \
  --name nominatim \
  mediagis/nominatim:4.2

# Find container id in another terminal, for this instance: cce4a525a8e2
docker ps

# Open another terminal and copy planet osm psf file into container
# Actually, this is not necessary, because I can use bind mounts (docker run -v)
#docker cp ~/data_nominatim/planet-230213.osm.pbf cce4a525a8e2:/nominatim/

# Start bash shell for the container and check if the file is there
docker exec -it cce4a525a8e2 bash
ls /nominatim/ # <--- confirmed

# Commit the container
docker commit -m "With planet data" -a "Shinhan Shiu" cce4a525a8e2 shius/nominatim_planet
docker images

# Save image
docker save shius/nominatim_planet > nominatim_planet.tar

# Stop and remove docker container
docker stop cce4a525a8e2
docker rm cce4a525a8e2

# Run this again to test local .pbf
docker run -it \
  -e PBF_PATH=/nominatim/data/monaco-latest.osm.pbf \
  -p 8080:8080  \
  -v /osm-maps/data:/nominatim/data \
  --name nominatim \
  mediagis/nominatim:4.2

docker run -it \
  -e PBF_PATH=/nominatim/planet-230213.osm.pbf \
  -p 8080:8080  \
  -v /osm-maps/data:/nominatim/ \
  --name nominatim_planet \
  shius/nominatim_planet
'''

In [ ]:
# Try michigan
'''bash
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/north-america/us/michigan-latest.osm.pbf \
  -p 8080:8080  \
  --name nominatim_mi \
  mediagis/nominatim:4.2
'''


#### [Aximem/nominatim-docker-multiple-regions](https://github.com/Aximem/nominatim-docker-multiple-regions)

Not well documented and it did not work for me.

In [ ]:
'''bash
# In /home/shius/github
git clone https://github.com/Aximem/nominatim-docker-multiple-regions.git
cd nominatim-docker-multiple-regions/

# Build image
docker build --pull --rm -t nominatim .

# update file: multiple_regions/init_multiple_regions.sh
# Issue: Not sure where to find this file.
#COUNTRIES="africa australia-oceania europe antarctica central-america south-america asia"

# init multiple regions: did not work...
docker run -t -v /home/shius/data_nominatim/:/data nominatim sh /app/multiple_regions/init.sh
'''

## ___Testing: local Nominatim install___

Using the Docker version I could just specify the different continent in different runs. But that is very clunky. So I am going to try install locally. Um... just too many dependent packages... Not doing this for now, stop at ICU install.

Following the [instruction](https://nominatim.org/release-docs/latest/admin/Installation/). More specifically, [for Ubuntu 20.04](https://nominatim.org/release-docs/latest/appendix/Install-on-Ubuntu-20/)


### Installing the Required Software



In [ ]:
'''
sudo apt update -qq
sudo apt install -y php-cgi
sudo apt install -y build-essential cmake g++ libboost-dev libboost-system-dev \
                    libboost-filesystem-dev libexpat1-dev zlib1g-dev \
                    libbz2-dev libpq-dev liblua5.3-dev lua5.3 \
                    postgresql-12-postgis-3 \
                    postgresql-contrib-12 postgresql-12-postgis-3-scripts \
                    php-cli php-pgsql php-intl libicu-dev python3-dotenv \
                    python3-psycopg2 python3-psutil python3-jinja2 \
                    python3-icu python3-datrie python3-yaml
'''

### System Configuration

#### Creating Dedicated User Accounts

In [ ]:
'''
# Nominatim will run as a global service on your machine. It is therefore best 
# to install it under its own separate user account
sudo useradd -d /srv/nominatim -s /bin/bash -m nominatim

# export user name and home directory 
export USERNAME=nominatim
export USERHOME=/srv/nominatim

# Make sure that system servers can read from the home directory
# NOTE: add sudo otherwide `Operation not permitted`, SHOULD NOT DO THIS...
sudo chmod a+x $USERHOME
'''

#### Setting up PostgreSQL

[Tuning the PostgreSQL database for nominatim](https://nominatim.org/release-docs/latest/admin/Installation/#tuning-the-postgresql-database)
- Use recommended values except for:
  - maintenance_work_mem = 5GB instead of 10GB

```
shared_buffers = 2GB
maintenance_work_mem = (10GB)
autovacuum_work_mem = 2GB
work_mem = (50MB)
effective_cache_size = (24GB)
synchronous_commit = off
max_wal_size = 1GB
checkpoint_timeout = 10min
checkpoint_completion_target = 0.9
autovacuum_max_workers = 1
fsync = off
full_page_writes = off
```

- Don't forget to re-enable them after the initial import:
```
fsync = on
full_page_writes = on
```

In [ ]:
'''
sudo vim /etc/postgresql/12/main/postgresql.conf
'''

In [ ]:
'''
# restart the postgresql service after updating this config file
sudo systemctl restart postgresql

# Get `System has not been booted with systemd as init system...`
# see: https://stackoverflow.com/questions/70332810/postgresql-problem-system-has-not-been-booted-with-systemd-as-init-system-pid

sudo service postgresql status
sudo service postgresql start

# Add two postgres users
# 1. user that does the import
# 2. another for the webserver
sudo -u postgres createuser -s $USERNAME
sudo -u postgres createuser www-data
'''

### Installing Nominatim

#### Building and Configuration

In [ ]:
'''
cd $USERHOME
wget https://nominatim.org/release/Nominatim-4.2.1.tar.bz2
tar xf Nominatim-4.2.1.tar.bz2
'''

## ___Testing: first try on export___

### Try to get nominatim running after exporting

On 3/14/23, found issues with the nominatin runs where some address should be US but was pointed to China. Found this for records with "New Heaven" but the State name and postal code should be used before that anyway, so not sure what's going on. Beacuse of this, I need to rerun North America nominatim. I generated an export of container after the database was built but run into issues about getting it to run again. Here are the things I have tried:

- Import container
  - `docker import nominatim_north-america.tar mediagis/nominatim:4.2`
- Just do a straight call like how I build the databse
  - Does not work, ask for command
- Add `/app/start.sh` as command
  - Does not work:
    - `sed: can't read /.env: No such file or directory`
    - `chown: missing operand after 'nominatim:nominatim`
- Try [this solution](https://github.com/mediagis/nominatim-docker/discussions/306)
- Also see [this](https://github.com/mediagis/nominatim-docker/discussions/280)
- And [this](https://github.com/mediagis/nominatim-docker/discussions/351)

Try persistent container
- [Nominatim instruction](https://github.com/mediagis/nominatim-docker/discussions/351)
- [Backup, restore, migrate data volumes](https://docs.docker.com/storage/volumes/#back-up-restore-or-migrate-data-volumes)

In [ ]:
# For testing
'''
# Check volume is created
docker volume ls

# Docker volume backup
#docker run --rm --volumes-from [CONTAINER] -v $(pwd):/backup ubuntu tar cvf /backup/backup.tar /dbdata
# Or
#docker run --rm \ 
#  --volume [DOCKER_COMPOSE_PREFIX]_[VOLUME_NAME]:/[TEMPORARY_DIRECTORY_TO_STORE_VOLUME_DATA] \
#  --volume $(pwd):/[TEMPORARY_DIRECTORY_TO_STORE_BACKUP_FILE] \
#  ubuntu \
#  tar cvf /[TEMPORARY_DIRECTORY_TO_STORE_BACKUP_FILE]/[BACKUP_FILENAME].tar /[TEMPORARY_DIRECTORY_TO_STORE_VOLUME_DATA]

docker run --rm \
  --volumes-from nominatim-test \
  -v $(pwd):/backup \
  busybox \
  tar cvf /backup/backup_andorra.tar /var/lib/postgresql/14/main

# Docker volume restore
#docker run -v /dbdata --name dbstore2 ubuntu /bin/bash
#docker run --rm --volumes-from dbstore2 -v $(pwd):/backup ubuntu bash -c "cd /dbdata && tar xvf /backup/backup.tar --strip 1"
# Or
# docker run --rm \ 
#  --volume [DOCKER_COMPOSE_PREFIX]_[VOLUME_NAME]:/[TEMPORARY_DIRECTORY_STORING_EXTRACTED_BACKUP] \
#  --volume $(pwd):/[TEMPORARY_DIRECTORY_TO_STORE_BACKUP_FILE] \
#  ubuntu \
#  tar xvf /[TEMPORARY_DIRECTORY_TO_STORE_BACKUP_FILE]/[BACKUP_FILENAME].tar -C /[TEMPORARY_DIRECTORY_STORING_EXTRACTED_BACKUP] --strip 1

#docker run -v /var/lib/postgresql/14/main --name nominatim-test_restore ubuntu /bin/bash
docker run --rm \
  -v nominatim-data-test2:/var/lib/postgresql/14/main \
  -v $(pwd):/backup \
  ubuntu \
  tar xvf /backup/backup-andorra.tar -C /var/lib/postgresql/14/main --strip 1


cd /home/shinhan/projects/plant_sci_hist/7_countries/testing
docker ps
docker export 191ccc5eecce > nominatim-test.tar

# Copy volume to local storage
#docker cp --archive CONTAINER:/MOUNT_POINT_OF_VOLUME ./LOCAL_FOLDER
docker cp --archive 0973395e062b:/var/lib/postgresql/14/main ./nominatim-data-test

# Import into another container
#### DOES NOT WORK ####
#docker run -v ./LOCAL_FOLDER:/MOUNT_POINT_OF_VOLUME some_image


#### This works but ####
## initdb: error: directory "/var/lib/postgresql/14/main" exists but is not empty
#docker volume create --name my_test_volume --opt type=none --opt device=/home/jinna/Jinna_Balu/Test_volume --opt o=bind
docker volume create \
  --name nominatim-data-test2 \
  --opt type=none \
  --opt device=/home/shinhan/projects/plant_sci_hist/7_countries/testing/nominatim-data-test \
  --opt o=bind

#### Try to add main toward the end ####
docker volume create \
  --name nominatim-data-test3 \
  --opt type=none \
  --opt device=/home/shinhan/projects/plant_sci_hist/7_countries/testing/nominatim-data-test/main \
  --opt o=bind

docker volume ls # check volume is created

docker run -it \
  -e PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data-test:/var/lib/postgresql/14/main \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim-import \
  mediagis/nominatim:4.2



# Export volume
docker run \
  --volumes-from nominatim-test \
  -v $(pwd):/backup \
  mediagis/nominatim:4.2 \
  tar cvf nominatim-data-test.tar /var/lib/postgresql/14/main


# Import container
cat nominatim_north-america.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded

# Import volume

'''


#### Run, export, stop, remove, then import

In the following, I :
- Run nominatim docker, download Andorra osm.pbf data, build database using the persistent container setting.
- Export the container with a docker volume setup
- Stop and remove the container, delete the loaded image.
- Dockerd is still running so the docker volume is still there.
  - The image created (nominatim_persistent:latest) is also still there.
- Import the exported container
- Run the imported image by passing proper variables.
  - Specifically: -e PROJ_DIR and -v

In [ ]:

'''docker run -it --shm-size=1g \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -v nominatim-data:/var/lib/postgresql/14/main \
  -p 8080:8080 \
  --name nominatim_andorra \
  mediagis/nominatim:4.2'''

# Export/import/run
'''
docker ps
docker volume ls    # Check volume exist
docker export -o test_persistent_andorra.tar 23d68084ca58

docker rm -f 23d68084ca58

docker ps -a        # check container is no longer there

# Load container
cat test_persistent_andorra.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded
'''

##### Solution 1: enter container and edit /app/start.sh

For a while, I am stuck when do docker run with `/app/start.sh` where the following error occurs:
- `chown: missing operand after 'nominatim:nominatim'`
- Looking into the shell script, the `chomn` commnad is followed by an environmental variable `${PROJECT_DIR}` which is not set.
- The following finally worked.

In [ ]:
# Solution 1
# Enter container and edit /app/start.sh
'''
# Run container
# Use /app/start.sh failed.
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_persistent:latest \
  /bin/bash
'''

# Enter container
'''
apt-get update
apt-get install vim
vim /app/start.sh

# In VIM
# Add a line before IMPORT_FINISHED
PROJECT_DIR=/nominatim

# Back in bash
/app/start.sh
'''

##### Solution 2: pass the environmental variable

This works also, much easier. Finally.

In [ ]:
# Enter container and edit /app/start.sh
'''
docker ps -a     # make sure the previous container has stopped
docker images    # confirm nominatim_persistent:latest exist
docker volume ls # confirm nominatim-data exist

docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -e PROJECT_DIR=/nominatim \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_persistent:latest \
  /app/start.sh
'''

#### Retry the above using a local osm.pbf file

To see if the following works:
- Set PBF_FILE environmental variable
- Set a bind mount for the osm.pbf data folder

All the above still works

In [ ]:
# Run container and create data volume: Ok
'''
docker run -it  \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -v nominatim-data-test:/var/lib/postgresql/14/main \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  -p 8080:8080 \
  --name nominatim_andorra \
  mediagis/nominatim:4.2

docker ps
docker volume ls    # Check volume exist
'''

# Export/import/run
'''
docker export -o test_persistent_andorra2.tar d6f6a0e256e6

docker rm -f 23d68084ca58

docker ps -a        # check container is no longer there

# Load container
cat test_persistent_andorra.tar | docker import - nominatim_persistent:latest
docker images       # A new image loaded
'''

#### Use an existing volume 

##### By starting an unrelated (i.e., different location) nominatim container

The naminatim-data volume is created using Andorra data, want to see if I start a container specifying a different location but using the Andorra volume, what happens:
- Delete images
- Run nominatim docker
  - Specify PBF_URL to download equatorial-guinea
  - Specify location of equatorial-guinea osm.pbf
  - Specify nominatim-data mount 
- Testing
  - Search Andorra returned value.
  - Search Malabo (Eq Guinea's captial) returned nothing
  - So this is using Andorra database, even though EG osm is specified.

In [ ]:
'''
# Remove image, this does not really matter since I am not using that docker
# container. Just to make this cleaner
docker images
docker image remove -f nominatim_persistent

docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_eg \
  mediagis/nominatim:4.2
'''

##### By using an imported data volume

Next I want to see if I can restore the data volume after exporting.
- See [this section](https://docs.docker.com/storage/volumes/#back-up-restore-or-migrate-data-volumes) on backup
- This seems complocated.

In [ ]:
# Run persistent container, building database with a local file
# -p setting doc:
# https://www.mend.io/free-developer-tools/blog/docker-expose-port/
'''
docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 2346:2346 \
  -v nominatim-data-test:/var/lib/postgresql/14/main \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim-test \
  mediagis/nominatim:4.2
'''

### The following does not work

#### Set up test donor and recipient nominatins

In [ ]:
# Get osm.pbf
'''
wget https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf
wget https://download.geofabrik.de/europe/andorra-latest.osm.pbf
'''

# Start recipient server
'''
docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v /home/shius/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_rec \
  mediagis/nominatim:4.2
'''

# Export/import/run
'''
docker export -o test_andorra.tar 47ad114454aa

docker stop 47ad114454aa
docker rm 47ad114454aa

cat test_andorra.tar | docker import - nominatim_andorra:latest

docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_rec \
  nominatim_andorra:latest \
  /app/start.sh
'''
# The above doe not work


In [ ]:
# Start donor server
'''
docker run -it \
  -e PBF_PATH=PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -e FREEZE=true \
  -p 808:8080 \
  -v ~/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  -v nominatim-data:/var/lib/postgresql/14/main \
  --name nominatim_don \
  mediagis/nominatim:4.2

docker ps

# Go to recipient container
docker exec -it e01ff686f768 /bin/bash
# In recipient container
cd /var/lib/postgresql/14
du main                                   # 327096  main
exit

# Go to doner container
docker exec -it 35c4c0cdfb93 /bin/bash
cd /var/lib/postgresql/14
du main                                   # 397388  main
exit
'''

#### Try 1: Copy database main folder over

In [ ]:
'''
# In host, copy donor postgresql database
docker cp 35c4c0cdfb93:/var/lib/postgresql/14/main ./  # copy files
du main/                                  # 397044  main/

# Test copy
mkdir -p test_dir/main
cp -r main/ test_dir/main/

# Replace recipient postgresql database
docker ps                                 # container id: e01ff686f768
docker cp main/ e01ff686f768:/var/lib/postgresql/14/

# Go to recipient container to change ownership
docker exec -it e01ff686f768 /bin/bash
chown -R postgres:postgres /var/lib/postgresql/14/main

# Test http://localhost:8080/search.php?q=usa
code: 500
Message: 'Failed to establish database connection:SQLSTATE[08006]

# Stop the server than restart
docker stop e01ff686f768
docker start e01ff686f768

# Cannot restart
'''

#### Try 2: Use a read-only bind mount

See read-only bind mount secton of [this](https://docs.docker.com/storage/bind-mounts/)
- This failed: `FATAL: Creating new database failed`

In [ ]:
# In ~/projects/plant_sci_hist/7_countries/testing
'''
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -p 8080:8080 \
  --name nominatim_rec \
  -v main:/var/lib/postgresql/14/main:ro \
  mediagis/nominatim:4.2
'''

#### Try 3: Use a bind mout that's writable

Rid of the `ro` parameter
- Does not work.
- The resulted database is overwritten.
- Or the mount has never worked because the wsl copy remain the same, just the folder in the container is not what it is supposed to be.

In [ ]:
# In ~/projects/plant_sci_hist/7_countries/testing
'''
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/europe/andorra-latest.osm.pbf \
  -p 8080:8080 \
  --name nominatim_rec \
  -v main:/var/lib/postgresql/14/main \
  mediagis/nominatim:4.2
'''

#### Try 4: pg_dump/pg_restore

https://nominatim.org/release-docs/latest/admin/Advanced-Installations/#moving-the-database-to-another-machine

## ___Testing: docker volumne backup, export, import try again (working)___

[This post](https://www.howtogeek.com/devops/how-to-back-up-your-docker-volumes/)
- But the import volume part did not work, lead to a new volume without any data imported using the commands shown.

### Create a container that stores data in a volume

https://stackoverflow.com/questions/69320727/exporting-a-docker-container-and-volume

In [ ]:
# Mysql example
docker run -d \
  --name mysql \
  -v mysql_data:/var/lib/mysql \
  -e MYSQL_ROOT_PASSWORD=mysql \
  mysql:8

docker volume ls
docker volume inspect mysql_data

In [ ]:
# Nominatim example
docker run -it \
  -e PBF_PATH=/nominatim/data/andorra-latest.osm.pbf \
  -e FREEZE=true \
  -p 8080:8080 \
  -v data_andorra:/var/lib/postgresql/14/main \
  -v /home/shinhan/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_andorra \
  mediagis/nominatim:4.2

docker volume ls
docker volume inspect data_andorra

# Committ/save/export container
docker commit nominatim_andorra nominatim_andorra-commit:latest
docker save nominatim_andorra-commit > nominatim_andorra-save.tar

#https://www.techrepublic.com/article/how-to-exportimport-containers-with-docker/
docker export nominatim_andorra > nominatim_andorra-export.tar

### Start a temporary container to back up the "mysql_data" volume

- Mount an existing container’s volumes into your backup container. 
- Deposit an archive of the volume’s content into your working directory.
- `--volumes-from`: the temporary backup container receives access to the mysql container’s volumes
- `/var/lib/mysql`: specify this director that is inside the backup container exposes the volume’s content because this is the path used by the mysql container.
- `--rm`: remove the backup container as soon as the command completes

In [ ]:
docker run --rm \
  --volumes-from mysql \
  -v $PWD:/backup \
  ubuntu tar cvf /backup/backup-mysql.tar /var/lib/mysql

In [ ]:
# Nominatim example
docker run --rm \
  --volumes-from nominatim_andorra \
  -v $PWD:/backup \
  busybox tar cvf /backup/backup-andorra.tar /var/lib/postgresql/14/main

# stop nominatim run
docker rm -f nominatim_andorra

### Restore backup to replace contents of an existing volume

So this exist already. Not my user case with nominatim.
- But I can create a temporary one volume to make it fit my use based on this:
  - https://docs.docker.com/storage/volumes/
- This generates a volume that works, but name is long and cluky
  - See [this post](https://stackoverflow.com/questions/73233336/rename-docker-volume-on-docker-desktop) about changing volume name.

In [ ]:
# establish a temp container
docker run -v /var/lib/mysql --name dbrestore ubuntu /bin/bash

# restore volume through the temp container
# The following does not work. 
# The part "cd ..." lead to a wrong directory structure
#docker run --rm \
#  --volumes-from dbrestore \
#  -v $PWD:/backup \
#  bash -c "cd /var/lib/mysql && tar xvf /backup/backup-mysql.tar"

# Remove "cd ..." works
docker run --rm \
  --volumes-from dbrestore \
  -v $PWD:/backup \
  bash -c "tar xvf /backup/backup-mysql.tar"

# Rename volume
docker volume ls
docker volume create --name mysql_data_restore

docker run --rm -it \
  -v c2d45eecc64d8f4bb608b2b3c73b73e023117738dfa821d61a4c4e1b8be0e7dd:/from \
  -v mysql_data_restore:/to \
  bash -c "cd /from ; cp -av . /to"

docker ps -a  # Check if there is container using the old volume
docker rm -f bbbe65
docker volume rm c2d45eecc64d8f4bb608b2b3c73b73e023117738dfa821d61a4c4e1b8be0e7dd


# This WORKSSS!!!!

In [ ]:
# For nominatim
# temp container: nomi_restore
docker run -v /var/lib/postgresql/14/main --name nomi_restore ubuntu /bin/bash

docker run --rm \
  --volumes-from nomi_restore \
  -v $PWD:/backup \
  bash -c "tar xvf /backup/backup-andorra.tar"

docker volume ls
docker volume create --name data_andorra_restore

docker run --rm -it \
  -v 3ae370e6338ab458bcbed92081d60cb332a65128dfd78397ae9db0be786cfd46:/from \
  -v data_andorra_restore:/to \
  bash -c "cd /from ; cp -av . /to"

#docker ps -a  # Check if there is container using the old volume
#docker rm -f 33fa297caa91
#docker volume rm 9333011fc3975fe5ce824f9011bf4e3820b28eb7c83541badbdeb15781511277

### [DOES NOT WORK] Restore backup to a new host

This does not work. The volume created is empty.

In [ ]:
# Does not work
docker volume create mysql_data_restore

docker run --rm \
  -v mysql_data_restore:/var/lib/mysql \
  -v $PWD:/backup \
  ubuntu tar cvf /backup/backup-mysql.tar /var/lib/mysql


In [ ]:
# Does not work
# For nomiantim
docker volume create data_andorra_restore

docker run --rm \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  -v $PWD:/backup \
  ubuntu tar cvf /backup/backup-andorra.tar /var/lib/postgresql/14/main


### Start app

In [ ]:
docker run -d \
    --name mysql2 \
    -v mysql_data_restore:/var/lib/mysql \
    -e MYSQL_ROOT_PASSWORD=mysql \
    mysql:8

### Start nominatim with different approaches

#### Start nominatim using the original docker image

In [ ]:
# Nominatim example, deliberately use Eq Guinea data to see if:
#   Database is not being rebuilt.
#   The data use is Andorra data not Eq Guinea.

###
# This does not work INITIALLY
###
#initdb: error: directory "/var/lib/postgresql/14/main" exists but is not empty
# Later on, I check the directory structure of imported volume and realize that
# it is compeltely wrong. Now it works.

docker run -it \
  -e PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -p 8080:8080 \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  -v /home/shinhan/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_andorra2 \
  mediagis/nominatim:4.2

########
##
## DID NOT TEST THE REST TO SEE IF THEY WORK OR NOT
##
#########

In [ ]:
###
# This does not work
###
#initdb: error: directory "/var/lib/postgresql/14/main" exists but is not empty
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf \
  -p 8080:8080 \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  --name nominatim_andorra3 \
  mediagis/nominatim:4.2

docker rm nominatim_andorra3

#### Start nominatim with a committed image

In [ ]:
###
# This does not work
###
#initdb: error: directory "/var/lib/postgresql/14/main" exists but is not empty
docker run -it \
  -e PBF_PATH=/nominatim/data/equatorial-guinea-latest.osm.pbf \
  -p 8080:8080 \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  -v /home/shinhan/projects/plant_sci_hist/7_countries/testing:/nominatim/data \
  --name nominatim_andorra4 \
  nominatim_andorra-commit:latest

#### Load the exported container then run it

- https://computingforgeeks.com/how-to-export-and-import-docker-images-containers/
- https://adamtheautomator.com/docker-save-image/

In [ ]:
###
# This does not work
###
#initdb: error: directory "/var/lib/postgresql/14/main" exists but is not empty

# Import container
docker import nominatim_andorra-export.tar nominatim_andorra-export:latest

docker run -it \
  -e PBF_URL=https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf \
  -e PROJECT_DIR=/nominatim \
  -p 8080:8080 \
  -v data_andorra_restore:/var/lib/postgresql/14/main \
  --name nominatim_andorra5 \
  nominatim_andorra-export:latest \
  /app/start.sh


In [ ]:
# Well, this works using the orignal volume... somehow the volume export did
# not work...

docker run -it \
  -e PBF_URL=https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf \
  -e PROJECT_DIR=/nominatim \
  -p 8080:8080 \
  -v data_andorra:/var/lib/postgresql/14/main \
  --name nominatim_andorra6 \
  nominatim_andorra-export:latest \
  /app/start.sh

#### Test if using original database name but based o copy works

- Change data_andorra to data_andorra_original
- Change data_andorra_restore to data_andorra
- Do nominatim again

In [ ]:
docker volume create --name data_andorra_original

docker run --rm -it \
  -v data_andorra:/from \
  -v data_andorra_original:/to \
  bash -c "cd /from ; cp -av . /to"

# Man, this still works
docker run -it \
  -e PBF_URL=https://download.geofabrik.de/africa/equatorial-guinea-latest.osm.pbf \
  -e PROJECT_DIR=/nominatim \
  -p 8080:8080 \
  -v data_andorra_original:/var/lib/postgresql/14/main \
  --name nominatim_andorra7 \
  nominatim_andorra-export:latest \
  /app/start.sh

#### Issues (resolved)

Was wondering why the original volume work but not the imported one
- Go into wsl as root to check the volume folders
- Compare the volume folders: see [this](https://linuxhandbook.com/compare-directories/)

Turn out the folder contents are very different with the exported one to be ~10x smaller.
- Checking the exported volume tar file, confirm it is the same as the volume data. So the probelm is during the restore step.

In [ ]:
# In CMD with admin
wsl -u root

# In wsl: user: root
cd /var/lib/docker/volumes
diff -qr data_andorra data_andorra_restore/
# Tons of differeneces

# user: shiu
cd ~/projects/plant_sci_hist/7_countries/testing/var/lib/postgresql/14/main
tar xvf backup-andorra.tar

# In wsl
cd /var/lib/docker/volumes/data_andorra/_data
diff -qr ./ /home/shinhan/projects/plant_sci_hist/7_countries/testing/var/lib/postgresq
l/14/main
# No difference

